<a href="https://colab.research.google.com/github/MechalTimotewos/FBOFAC_Deep_Learning_Lab_Assignment/blob/main/04_hyperopt_wandb_updated_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copyright

<PRE>
This notebook was created as part of the "Deep learning / VITMMA19" class at
Budapest University of Technology and Economics, Hungary,
https://portal.vik.bme.hu/kepzes/targyak/VITMMA19

Any re-use or publication of any part of the notebook is only allowed with the
 written consent of the authors.

2024 (c) Mohammed Salah Al-Radhi and Tamás Gábor Csapó (malradhi@tmit.bme.hu)
</PRE>

In [1]:
### HYPEROPT: task during the class - we will do this together
# add WandB.ai integration to the code
# (help: https://docs.wandb.ai/guides/integrations/lightning )
# run at least 3 different trainings

In [2]:
# install pytorch lithening
!pip install pytorch-lightning --quiet
!pip install wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00


In [3]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10
from pytorch_lightning.loggers import WandbLogger
import wandb


In [4]:
# create one class to deal with data
class CifarDataModule(pl.LightningDataModule):
  def __init__(self, batch_size, data_dir="./"):
    super().__init__()
    self.data_dir=data_dir
    self.batch_size=batch_size
    self.transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    self.num_classes=10

  def prepare_data(self):
    CIFAR10(self.data_dir,train=True,download=True)
    CIFAR10(self.data_dir,train=False,download=True)

  def setup(self, stage=None):
    if stage=='fit' or stage is None:
      cifar_full=CIFAR10(self.data_dir,train=True,transform=self.transform)
      self.cifar_train,self.cifar_val=random_split(cifar_full,[45000,5000])

    if stage=='test' or stage is None:
      self.cifar_test=CIFAR10(self.data_dir,train=False,transform=self.transform)

  def train_dataloader(self):
    return DataLoader(self.cifar_train,batch_size=self.batch_size,shuffle=True,num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.cifar_val,batch_size=self.batch_size,shuffle=False,num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.cifar_test,batch_size=self.batch_size,shuffle=False,num_workers=2)

In [5]:
class CIFAR10LitModel(pl.LightningModule):
    def __init__(self, input_shape,num_classes,learning_rate=3e-4):
      super().__init__()
      self.save_hyperparameters()
      self.input_shape=input_shape
      self.learning_rate=learning_rate

      # model architecture
      self.conv1=nn.Conv2d(3,32,3,1)
      self.conv2=nn.Conv2d(32,32,3,1)
      self.conv3=nn.Conv2d(32,64,3,1)
      self.conv4=nn.Conv2d(64,64,3,1)
      self.pool1=nn.MaxPool2d(2)
      self.pool2=nn.MaxPool2d(2)

      n_sizes = self._get_output_shape(input_shape)
      self.fc1=nn.Linear(n_sizes,512)
      self.fc2=nn.Linear(512,128)
      self.fc3=nn.Linear(128,num_classes)

      self.train_acc=Accuracy(task='multiclass',num_classes=10)
      self.val_acc=Accuracy(task='multiclass',num_classes=10)
      self.test_acc=Accuracy(task='multiclass',num_classes=10)


    def _get_output_shape(self, shape):
          '''returns the size of the output tensor from the conv layers'''
          batch_size = 1
          input = torch.autograd.Variable(torch.rand(batch_size, *shape))
          output_feat = self._feature_extractor(input)
          n_size = output_feat.data.view(batch_size, -1).size(1)
          return n_size


  # conv1,relu, conv2,relu, maxpool,conv3,relu,conv4,relu,maxpool
    def _feature_extractor(self,x):
      x=F.relu(self.conv1(x))
      x=self.pool1(F.relu(self.conv2(x)))
      x=F.relu(self.conv3(x))
      x=self.pool2(F.relu(self.conv4(x)))
      return x


    def forward(self,x):
      x=self._feature_extractor(x)
      x=x.view(x.size(0),-1)
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      x=F.log_softmax(self.fc3(x),dim=1)
      return x

    def training_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      # metric
      preds = torch.argmax(logits, dim=1)
      acc = self.train_acc(preds, y)
      self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
      return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      preds = torch.argmax(logits, dim=1)
      acc = self.val_acc(preds, y)
      self.log('val_loss', loss, prog_bar=True)
      self.log('val_acc', acc, prog_bar=True)
      return loss

    # test loop
    def test_step(self,batch,batch_idx):
      x,y=batch
      logits=self(x)
      loss=F.nll_loss(logits,y)

      pred=torch.argmax(logits,dim=1)
      acc=self.test_acc(pred,y)
      self.log('test_loss',loss,on_epoch=True)
      self.log('test_acc',acc,on_epoch=True)
      return loss

    def configure_optimizers(self):
      optimizer=torch.optim.Adam(self.parameters(),self.learning_rate)
      return optimizer


In [6]:
# class for visualizing one batch of validation images along with predicted and rall class label
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, 1)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                            for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
            })

In [7]:
# Instantiate the cifar and model
cifar = CifarDataModule(batch_size=32)
cifar.prepare_data()
cifar.setup()

# Grab samples to log predictions on
samples = next(iter(cifar.val_dataloader()))

100%|██████████| 170498071/170498071 [00:03<00:00, 48906581.58it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [8]:
### WandB, you have have an account(if you don't, create one)
def train_model(learning_rate=1e-3):
    wandb.login(key='')
    config=wandb.config
    wandb_logger = WandbLogger(project='lastt', job_type='train', log_model="all")
    # instantiate classes
    dm = CifarDataModule(32)
    dm.prepare_data()
    dm.setup()
    model = CIFAR10LitModel((3, 32, 32), dm.num_classes, learning_rate)
    wandb_logger.watch(model)
    # Initialize Callbacks
    checkpoint_callback = pl.callbacks.ModelCheckpoint()
    early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")
    ### WandB
    trainer = pl.Trainer(max_epochs=5,
                     logger=wandb_logger,
                     callbacks=[checkpoint_callback, early_stop_callback,ImagePredictionLogger(samples)]
                    )
    # Train the model
    trainer.fit(model, dm)

    # Evaluate the model
    trainer.test(dataloaders=cifar.test_dataloader())
    # tell the WandB you have finished
    wandb.finish()

In [9]:
# Start training the model
train_model()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at ./lastt/8nab1gh7/checkpoints/epoch=4-step=7035.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at ./lastt/8nab1gh7/checkpoints/epoch=4-step=7035.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7303000092506409     │
│         test_loss         │    0.8020454049110413     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▇▇▇▇▇▇▇█
global_step,▁▂▄▅▇█
test_acc,▁
test_loss,▁
train_acc_epoch,▁▅▆▇█
train_acc_step,▃▂▁▃▄▆▅▃▅▆▅▇▇▄▄▄▆▇▇▇▆▇▇▇▆▆▇▆▆▇▇▆▇█▇▇█▇█▇
train_loss_epoch,█▅▃▂▁
train_loss_step,▇█▆▆▆▆▆▆▅▅▆▅▅▄▄▃▃▃▄▃▃▃▃▄▃▁▃▂▄▂▂▂▄▂▂▄▂▄▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████
val_acc,▁▅▇██
val_loss,█▄▃▁▁


In [ ]:
### TASK OF THE STUDENT

# extend WandB.ai integration in the code with sweeps
# (e.g. add variables like learning rate, optimizer, neurons_FC1, neurons_FC2)
# help: https://docs.wandb.ai/guides/sweeps and
#       https://github.com/wandb/wandb/issues/5003
# store the hyperparameters and val_acc to wandb
# run at least 10 trainings
# in wandb.ai, export the result of the runs as a .csv file,
# in wandb.ai, create a report from the sweep results and share it by submitting
# the link in Moodle.

In [11]:
#By Gibiso Mechal Timotewos
#Neptune COde: FBOFAC

In [12]:
#sweep configuration
sweep_configuration = {
    'method': 'random',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
        },
    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.0005, 0.0001]
            },
        'optimizer': {
            'values': ['adam', 'sgd']
            },
        'neurons_FC1': {
            'values': [64, 128, 256]
            },
        'neurons_FC2': {
            'values': [32, 64, 128]
            },
        'batch_size': {
            'values': [32, 64]
            }
        }
    }




In [13]:
# Create the sweep ID
sweep_id = wandb.sweep(sweep=sweep_configuration, project="my-cifar10-project")

Create sweep with ID: buv0gnxm
Sweep URL: https://wandb.ai/mechal-timotewos-budapest-university-of-technology-and-e/my-cifar10-project/sweeps/buv0gnxm


In [14]:

class LitCifarModel(pl.LightningModule):
    def __init__(self, neurons_FC1, neurons_FC2, learning_rate, optimizer):
        super().__init__()
        self.save_hyperparameters()

        #Define the model
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*32*3, neurons_FC1),
            nn.ReLU(),
            nn.Linear(neurons_FC1, neurons_FC2),
            nn.ReLU(),
            nn.Linear(neurons_FC2, 10)
        )

        #Define the loss function
        self.criterion = nn.CrossEntropyLoss()


      # Define the accuracy metric, specifying task and number of classes
        self.accuracy = Accuracy(task='multiclass', num_classes=10)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log('train_loss', loss)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log('val_loss', loss)
        self.log('val_acc', acc, on_step=False, on_epoch=True)
        return loss


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.test_accuracy(y_hat, y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        return loss

    def configure_optimizers(self):
        if self.hparams.optimizer == 'adam':
            return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        elif self.hparams.optimizer == 'sgd':
            return torch.optim.SGD(self.parameters(), lr=self.hparams.learning_rate, momentum=0.9)


In [15]:

def train():
    # Initialize wandb run
    run = wandb.init()
    #run = wandb.init(project="my-cifar10-project")
    wandb_logger = WandbLogger(project="my-cifar10-project")

    # Set up data
    data_module = CifarDataModule(batch_size=wandb.config.batch_size)

    # Model setup
    model = LitCifarModel(
        neurons_FC1=wandb.config.neurons_FC1,
        neurons_FC2=wandb.config.neurons_FC2,
        learning_rate=wandb.config.learning_rate,
        optimizer=wandb.config.optimizer
    )

    # Trainer
    trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)
    trainer.fit(model, datamodule=data_module)

# Start the WandB sweep
wandb.agent(sweep_id, function=train, count=10)


wandb: Agent Starting Run: z0eyifwp with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.001
wandb: 	neurons_FC1: 128
wandb: 	neurons_FC2: 128
wandb: 	optimizer: adam


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | Sequential         | 411 K  | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | accuracy  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
411 K     Trainable params
0         Non-trainable params
411 K     Total params
1.645     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
wandb: WARNING Config item 'neurons_FC1' was locked by 'sweep' (ignored update).
wa

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇████
train_acc,▁▃▄▅▆▆▇▇██
train_loss,█▅▆▄▅▃▄▃▄▃▄▃▄▆▃▄▅▄▂▃▄▃▅▄▄▄▄▂▃▂▂▂▂▁▂▂▁▂▃▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
val_acc,▁▄▅▆▇████▇
val_loss,█▅▃▂▁▁▂▂▃▄
epoch,9
train_acc,0.66429
train_loss,0.87437
trainer/global_step,14069
val_acc,0.5208
